In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import xarray as xr
import os

In [ ]:
import dask
from dask import array as da
from dask.distributed import Client

In [ ]:
mode, quick = 'local', True
local_mem = 8

if mode in ('local', 'inproc'):
    dask.config.set({"distributed.dashboard.link": (
    os.environ.get('JUPYTERHUB_SERVICE_PREFIX', '/') +
    "proxy/{port}/status"
    )});
    nworkers = 1
    opts = dict(n_workers=nworkers,
                threads_per_worker=1,
                memory_limit='{:d}Gb'.format(local_mem//nworkers),
               )
        
if mode == 'inproc':
    client = Client(**opts,
                    processes=False)
elif mode == 'local':
    client = Client(**opts,
                    processes=True)
elif mode == 'remote':
    client = Client('tcp://dask-datacube-dask.labs:8786')
else:
    client = None
    
client

In [ ]:
from datacube import Datacube
from datacube.utils.geometry import CRS
from datacube.api.core import output_geobox
from datacube.storage.masking import make_mask
from odc.ui import show_datasets
from datacube.utils.rio import set_default_rio_config, activate_from_config

def worker_setup():
    # these settings will be applied in every worker thread
    set_default_rio_config(aws={'region_name': 'us-west-2'},
                           cloud_defaults=True)

dc = Datacube()

if client is not None:
    # Runs once on every worker process, not per worker thread!
    client.register_worker_callbacks(setup=worker_setup)

set_default_rio_config(aws={'region_name': 'us-west-2'},
                           cloud_defaults=True)
activate_from_config()

In [ ]:
dss = dc.find_datasets(product='ls8_usgs_wofs_scene',
                       lon=(100, 180), lat=(0, -90), # Australian only
                       )
len(dss)

In [ ]:
m = show_datasets(dss, 
                  style={'opacity': 0.3, 'fillOpacity': 0},
                  width='400px', 
                  height='400px', 
                  # zoom=7,
                  scroll_wheel_zoom=False)
m

In [ ]:
all_epsgs = set(ds.crs.epsg for ds in dss)
assert len(all_epsgs) == 1
the_epsg, = all_epsgs
crs = CRS('epsg:{}'.format(the_epsg))

gbox = output_geobox(output_crs=crs,
              resolution=(-30, 30), 
              align=(15, 15), 
              datasets=dss)
gbox

In [ ]:
srcs = dc.group_datasets(dss, 'time')
if quick:
    srcs = srcs[:4]
    
srcs.time.shape, len(dss)

In [ ]:
xx = dc.load_data(srcs, gbox, dss[0].type.measurements,
                  dask_chunks={'x': gbox.width, 
                               'y': gbox.height})
xx.water.data

In [ ]:
xx_wet = make_mask(xx.water, wet=True).sum(dim='time', dtype='uint16')
xx_dry = make_mask(xx.water, dry=True).sum(dim='time', dtype='uint16')
xx_clear = xx_wet + xx_dry
xx_freq = xx_wet.astype('float32')/xx_clear

display(xx_wet.data)
display(xx_freq.data)

In [ ]:
xx_freq.data.visualize(rankdir='LR') if quick else None

In [ ]:
%%time
xx_wet, xx_dry, xx_clear, xx_freq = da.compute(xx_wet, xx_dry, xx_clear, xx_freq)

In [ ]:
fig,ax = plt.subplots(2, 2, figsize=(14,14))

for ax, im, t in zip(ax.ravel(),
                     [xx_wet, xx_dry, xx_clear, xx_freq],
                     ['Wet', 'Dry', 'Clear', 'Freq']):
    ax.imshow(im.values[::4, ::4],
              interpolation='bicubic')
    ax.xaxis.set_visible(False)
    ax.yaxis.set_visible(False)
    ax.set_title(t)

---------------------------------------------------------